Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.linear_algebra</a>
========

In [ ]:
import proveit
from proveit import defaults
from proveit import a, b, c, X, Y, Z
from proveit.logic import InSet, InClass, CartExp
from proveit.numbers import Real, three
from proveit.linear_algebra import VecSpaces, VecAdd, ScalarMult
%begin demonstrations

### Vector space closure proofs under addition and scalar multiplication

In [ ]:
R3 = CartExp(Real, three)

In [ ]:
defaults.assumptions = [InSet(a, Real), InSet(b, Real), InSet(c, Real),
                        InSet(X, R3), InSet(Y, R3), InSet(Z, R3)]

In [ ]:
InSet(ScalarMult(a, X), R3).prove()

In [ ]:
InSet(VecAdd(X, Y, Z), R3).prove()

In [ ]:
lin_comb = VecAdd(ScalarMult(a, X), ScalarMult(b, Y), ScalarMult(c, Z))

In [ ]:
InSet(lin_comb, R3).prove()

In [ ]:
%end demonstrations